# Otimização de Rotas com Algoritmo Genético

Neste notebook vamos resolver o problema clássico de **otimização de rotas**,
também conhecido como **Problema do Caixeiro Viajante (TSP)**.
O objetivo é encontrar a ordem de visita das cidades que **minimiza** a distância total percorrida.

Vamos seguir um passo a passo semelhante ao usado na otimização de portfólio.

## Dados de Entrada

Definimos um conjunto de cidades com suas coordenadas em um plano cartesiano.

In [ ]:
import numpy as np

# Coordenadas (x, y) de cada cidade
cidades = {
    'A': (0, 0),
    'B': (1, 5),
    'C': (5, 2),
    'D': (6, 6),
    'E': (8, 3)
}

nomes = list(cidades.keys())

def dist(a, b):
    ax, ay = cidades[a]; bx, by = cidades[b]
    return np.hypot(ax - bx, ay - by)

dist_matrix = np.array([[dist(i, j) for j in nomes] for i in nomes])
dist_matrix

## Função de Avaliação (Distância Total)

A função de fitness será a distância total do percurso fechado (voltando à cidade inicial).

In [ ]:
def avaliacao(caminho):
    distancia = 0.0
    for i in range(len(caminho)):
        a = caminho[i]
        b = caminho[(i + 1) % len(caminho)]
        distancia += dist(a, b)
    return distancia

# Exemplo
avaliacao(nomes)

## Parâmetros do Algoritmo Genético

In [ ]:
pop_size = 50
geracoes = 200
taxa_mutacao = 0.1

## Geração da População Inicial

In [ ]:
import random

def gerar_populacao(tamanho):
    pop = []
    for _ in range(tamanho):
        individuo = nomes.copy()
        random.shuffle(individuo)
        pop.append(individuo)
    return pop

populacao = gerar_populacao(pop_size)
len(populacao)

## Função de Seleção

In [ ]:
def selecao(pop, fitness):
    prob = np.exp(-np.array(fitness))
    prob /= prob.sum()
    idx = np.random.choice(len(pop), size=len(pop), p=prob)
    return [pop[i] for i in idx]

## Função de Crossover

In [ ]:
def crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    child = p1[a:b] + [c for c in p2 if c not in p1[a:b]]
    return child

## Função de Mutação

In [ ]:
def mutacao(ind):
    if random.random() < taxa_mutacao:
        i, j = sorted(random.sample(range(len(ind)), 2))
        ind[i], ind[j] = ind[j], ind[i]
    return ind

## Algoritmo Genético

In [ ]:
def algoritmo_genetico():
    pop = gerar_populacao(pop_size)
    melhor = None
    melhor_fit = float('inf')
    historico = []
    for _ in range(geracoes):
        fitness = [avaliacao(ind) for ind in pop]
        best_idx = int(np.argmin(fitness))
        if fitness[best_idx] < melhor_fit:
            melhor_fit = fitness[best_idx]
            melhor = pop[best_idx]
        historico.append(melhor_fit)
        selecionados = selecao(pop, fitness)
        nova_pop = []
        for i in range(0, pop_size, 2):
            pai1 = selecionados[i]
            pai2 = selecionados[(i+1)%pop_size]
            f1 = mutacao(crossover(pai1, pai2))
            f2 = mutacao(crossover(pai2, pai1))
            nova_pop.extend([f1, f2])
        pop = nova_pop[:pop_size]
    return melhor, historico

melhor_rota, historico = algoritmo_genetico()
melhor_rota, historico[-1]

## Visualização do Resultado

In [ ]:
import matplotlib.pyplot as plt

plt.plot(historico)
plt.xlabel('Geração')
plt.ylabel('Distância')
plt.title('Evolução da Distância')
plt.show()

x = [cidades[c][0] for c in melhor_rota + [melhor_rota[0]]]
y = [cidades[c][1] for c in melhor_rota + [melhor_rota[0]]]
plt.figure()
plt.plot(x, y, 'o-')
for c in melhor_rota:
    plt.text(cidades[c][0], cidades[c][1], c)
plt.title('Melhor Rota Encontrada')
plt.show()

## Conclusão

Utilizando o algoritmo genético, conseguimos uma boa aproximação para o problema de otimização de rotas.